<a href="https://colab.research.google.com/github/Maneekul/Intern/blob/main/CmalNet_Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

drive  sample_data


In [ ]:
%cd /content/drive/MyDrive/CMAL-main

/content/drive/MyDrive/CMAL-main


In [ ]:
!python utils.py

In [ ]:
!pip install inplace-abn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.3/137.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for inplace-abn: filename=inplace_abn-1.1.0-cp310-cp310-linux_x86_64.whl size=5469863 sha256=c76c12c4e3659d264c9f9e92d4c05d45eee09106cd52db2189f469d57dd7fe92
  Stored in directory: /root/.cache/pip/wheels/c7/f2/05/97ab74dbf2ed6d592684f76d0ae8f33f66bbafa0ec5b53b416
Successfully built inplace-abn


In [ ]:
!pip install --upgrade torchvision


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tarfile
import os
import csv
import time
import torch
import json
from random import randint
from pathlib import Path
from PIL import Image
import PIL.Image
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision.models as models
from matplotlib.ticker import FormatStrFormatter
from google.colab.patches import cv2_imshow
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
from google.colab import drive
from IPython.display import Image
%matplotlib inline


In [ ]:
# Path to the tar file in your Google Drive
tar_file_path = '/content/drive/MyDrive/cars_standford/cars_train.tgz'




# Path to the folder where you want to extract the contents
extract_folder_path = '/tmp/cars_train.tgz/'

# Create the extract folder if it doesn't already exist
os.makedirs(extract_folder_path, exist_ok=True)

# Extract the contents of the tar file to the extract folder
with tarfile.open(tar_file_path, 'r') as tar_ref:
    tar_ref.extractall(extract_folder_path)


In [ ]:
# Path to the tar file in your Google Drive
tar_file_path = '/content/drive/MyDrive/cars_standford/cars_test.tgz'

# Path to the folder where you want to extract the contents
extract_folder_path = '/tmp/cars_test.tgz/'

# Create the extract folder if it doesn't already exist
os.makedirs(extract_folder_path, exist_ok=True)

# Extract the contents of the tar file to the extract folder
with tarfile.open(tar_file_path, 'r') as tar_ref:
    tar_ref.extractall(extract_folder_path)

In [ ]:
# Path to the tar file in your Google Drive
tar_file_path = '/content/drive/MyDrive/cars_standford/car_devkit.tgz'

# Path to the folder where you want to extract the contents
extract_folder_path = '/tmp/car_devkit.tgz'

# Create the extract folder if it doesn't already exist
os.makedirs(extract_folder_path, exist_ok=True)

# Extract the contents of the tar file to the extract folder
with tarfile.open(tar_file_path, 'r') as tar_ref:
    tar_ref.extractall(extract_folder_path)

In [ ]:
#Classes are assigned numerical value of 1-196, each
#mapping to a string. Data from cars_meta matlab file
#This applies to all data.
meta = loadmat('/tmp/car_devkit.tgz/devkit/cars_meta.mat')
meta.keys()
type(meta['class_names']),meta['class_names'].shape
[item.flat[0] for item in meta['class_names'][0][0]]
classes = [[row.flat[0] for row in line] for line in meta['class_names'][0]]
classDict = {}
classCount = 1
for c in classes:
    classDict.update({classCount:c[0]})
    classCount = classCount + 1
term = 0
for num, model in classDict.items():
    print('Class #', num,'Model String:', model)
    term = term + 1
    if term == 4:
        break
print('...')


Class # 1 Model String: AM General Hummer SUV 2000
Class # 2 Model String: Acura RL Sedan 2012
Class # 3 Model String: Acura TL Sedan 2012
Class # 4 Model String: Acura TL Type-S 2008
...


In [ ]:
#LABELS FOR TRAINING DATA
#contains bbox, class, and filename
labelfile = loadmat('/tmp/car_devkit.tgz/devkit/cars_train_annos.mat')
labelfile.keys()
type(labelfile['annotations']),labelfile['annotations'].shape
type(labelfile['annotations'][0][0]),labelfile['annotations'][0][0].shape
labelfile['annotations'][0][0]['bbox_x1'], labelfile['annotations'][0][0]['fname']
[item.flat[0] for item in labelfile['annotations'][0][0]]
"""
The following variable 'trainLabels' contains a list of labels for
every test image the format of each element is as follows:
### bbox_x1, bbox_y1, bbox_x2, bbox_y2, class, filename
"""
trainLabels = [[row.flat[0] for row in line] for line in labelfile['annotations'][0]]
#Outputting the table for readibility with pandas
columns = ['boundingbox_x1', 'boundingbox_y1', 'boundingbox_x2', 'boundingbox_y2', 'class', 'fname']
view = pd.DataFrame(trainLabels, columns=columns)
view.head()


,boundingbox_x1,boundingbox_y1,boundingbox_x2,boundingbox_y2,class,fname
0,39,116,569,375,14,00001.jpg
1,36,116,868,587,3,00002.jpg
2,85,109,601,381,91,00003.jpg
3,621,393,1484,1096,134,00004.jpg
4,14,36,133,99,106,00005.jpg


In [ ]:
#LABELS FOR TESTING DATA
#Contains bbox and filename
lf = loadmat('/tmp/car_devkit.tgz/devkit/cars_test_annos.mat')
lf.keys()
type(lf['annotations']),lf['annotations'].shape
type(lf['annotations'][0][0]),lf['annotations'][0][0].shape
lf['annotations'][0][0]['bbox_x1'], lf['annotations'][0][0]['fname']
[item.flat[0] for item in lf['annotations'][0][0]]
"""
The following variable 'testLabels' contains a list of labels for
every test image the format of each element is as follows:
### bbox_x1, bbox_y1, bbox_x2, bbox_y2, filename
there is no class name in this set of labels
"""
testLabels = [[row.flat[0] for row in line] for line in lf['annotations'][0]]

#Test labels class and fname
with open('/content/drive/MyDrive/csv_label/anno_test.csv') as csv_file:
    read = csv.reader(csv_file, delimiter=',')
    testDict = {}
    classCount = 1
    for row in read:
        testDict.update({row[0]:row[5]})
        classCount = classCount + 1
    print(testDict)
    print(len(testDict), 'processed')

{'00001.jpg': '181', '00002.jpg': '103', '00003.jpg': '145', '00004.jpg': '187', '00005.jpg': '185', '00006.jpg': '78', '00007.jpg': '118', '00008.jpg': '165', '00009.jpg': '32', '00010.jpg': '60', '00011.jpg': '49', '00012.jpg': '108', '00013.jpg': '116', '00014.jpg': '135', '00015.jpg': '83', '00016.jpg': '51', '00017.jpg': '154', '00018.jpg': '33', '00019.jpg': '22', '00020.jpg': '32', '00021.jpg': '151', '00022.jpg': '4', '00023.jpg': '81', '00024.jpg': '107', '00025.jpg': '165', '00026.jpg': '191', '00027.jpg': '170', '00028.jpg': '195', '00029.jpg': '91', '00030.jpg': '5', '00031.jpg': '164', '00032.jpg': '148', '00033.jpg': '188', '00034.jpg': '44', '00035.jpg': '33', '00036.jpg': '7', '00037.jpg': '31', '00038.jpg': '74', '00039.jpg': '30', '00040.jpg': '166', '00041.jpg': '180', '00042.jpg': '106', '00043.jpg': '3', '00044.jpg': '65', '00045.jpg': '191', '00046.jpg': '35', '00047.jpg': '75', '00048.jpg': '85', '00049.jpg': '25', '00050.jpg': '168', '00051.jpg': '137', '00052.j

In [ ]:
"""
Preprocess Training Images
Gets all training images and crops by bounding box, saves to a new directory
The directory housing training image has subdirectories named by class name
with each corresponding image in the corresponding folder
Saves ~7% of images for validation
"""
print(len(trainLabels), "images to process.")
Path("/tmp/processedTrain").mkdir(parents=True, exist_ok=True)
Path("/tmp/processedValidation").mkdir(parents=True, exist_ok=True)
count = 1
for l in trainLabels:
    print('Processing image', count, 'of', len(trainLabels))
    #bbox
    x1 = l[0]
    y1 = l[1]
    x2 = l[2]
    y2 = l[3]
    class_num = l[4]
    filename = l[5]
    address = "/tmp/cars_train.tgz/cars_train/" + filename
    classAppend = classDict.get(class_num) + '/'
    if count < 7000:
        saveDir = "/tmp/processedTrain/" + classAppend
    else:
        saveDir = "/tmp/processedValidation/" + classAppend
    im = PIL.Image.open(address)
    region = im.crop((x1, y1, x2, y2))
    Path(saveDir).mkdir(parents=True, exist_ok=True)
    saveAddress = saveDir + filename
    region.save(saveAddress)
    print('Saved', saveAddress)
    count = count + 1
print("------------------------")
print("Training images preprocessing complete.")

Streaming output truncated to the last 5000 lines.
Processing image 5646 of 8144
Saved /tmp/processedTrain/Chevrolet HHR SS 2010/05646.jpg
Processing image 5647 of 8144
Saved /tmp/processedTrain/Audi TT RS Coupe 2012/05647.jpg
Processing image 5648 of 8144
Saved /tmp/processedTrain/Bentley Arnage Sedan 2009/05648.jpg
Processing image 5649 of 8144
Saved /tmp/processedTrain/Ford F-150 Regular Cab 2007/05649.jpg
Processing image 5650 of 8144
Saved /tmp/processedTrain/BMW 3 Series Sedan 2012/05650.jpg
Processing image 5651 of 8144
Saved /tmp/processedTrain/Fisker Karma Sedan 2012/05651.jpg
Processing image 5652 of 8144
Saved /tmp/processedTrain/MINI Cooper Roadster Convertible 2012/05652.jpg
Processing image 5653 of 8144
Saved /tmp/processedTrain/Dodge Dakota Crew Cab 2010/05653.jpg
Processing image 5654 of 8144
Saved /tmp/processedTrain/BMW 1 Series Coupe 2012/05654.jpg
Processing image 5655 of 8144
Saved /tmp/processedTrain/Ford Freestar Minivan 2007/05655.jpg
Processing image 5656 of 81

In [ ]:
"""
Preprocess Testing Images
Gets all training images and crops by bounding box, saves to a new directory
"""
print(len(testLabels), "images to process.")
Path("/tmp/processedTest").mkdir(parents=True, exist_ok=True)
Path("/tmp/boundedTest").mkdir(parents=True, exist_ok=True)
count = 1
for l in testLabels:
    print('Processing image', count, 'of', len(testLabels))
    x1 = l[0]
    y1 = l[1]
    x2 = l[2]
    y2 = l[3]
    filename = l[4]
    c_num = testDict.get(filename)
    classAppend = classDict.get(int(c_num)) + '/'
    address = "/tmp/cars_test.tgz/cars_test/" + filename
    saveDir = "/tmp/processedTest/" + classAppend
    im = PIL.Image.open(address)
    region = im.crop((x1, y1, x2, y2))
    Path(saveDir).mkdir(parents=True, exist_ok=True)
    saveAddress = saveDir + filename
    saveAddress2 = "/tmp/boundedTest/" + filename
    region.save(saveAddress2)
    print('Saved', saveAddress2)
    region.save(saveAddress)
    print('Saved', saveAddress)
    count = count + 1
print("------------------------")
print("Testing images preprocessing complete.")


Streaming output truncated to the last 5000 lines.
Processing image 6376 of 8041
Saved /tmp/boundedTest/06376.jpg
Saved /tmp/processedTest/Rolls-Royce Phantom Sedan 2012/06376.jpg
Processing image 6377 of 8041
Saved /tmp/boundedTest/06377.jpg
Saved /tmp/processedTest/Dodge Charger SRT-8 2009/06377.jpg
Processing image 6378 of 8041
Saved /tmp/boundedTest/06378.jpg
Saved /tmp/processedTest/Dodge Caravan Minivan 1997/06378.jpg
Processing image 6379 of 8041
Saved /tmp/boundedTest/06379.jpg
Saved /tmp/processedTest/Audi TT RS Coupe 2012/06379.jpg
Processing image 6380 of 8041
Saved /tmp/boundedTest/06380.jpg
Saved /tmp/processedTest/Nissan 240SX Coupe 1998/06380.jpg
Processing image 6381 of 8041
Saved /tmp/boundedTest/06381.jpg
Saved /tmp/processedTest/Hyundai Sonata Sedan 2012/06381.jpg
Processing image 6382 of 8041
Saved /tmp/boundedTest/06382.jpg
Saved /tmp/processedTest/Bentley Arnage Sedan 2009/06382.jpg
Processing image 6383 of 8041
Saved /tmp/boundedTest/06383.jpg
Saved /tmp/processe

In [ ]:
!python train_Stanford_Cars_TResNet_L.py

Streaming output truncated to the last 5000 lines.
Step: 379 | Loss: 0.174 |Combined Acc: 95.000% (1083/1140)
Step: 380 | Loss: 0.174 |Combined Acc: 95.013% (1086/1143)
Step: 381 | Loss: 0.173 |Combined Acc: 95.026% (1089/1146)
Step: 382 | Loss: 0.173 |Combined Acc: 95.039% (1092/1149)
Step: 383 | Loss: 0.175 |Combined Acc: 94.965% (1094/1152)
Step: 384 | Loss: 0.175 |Combined Acc: 94.978% (1097/1155)
Step: 385 | Loss: 0.174 |Combined Acc: 94.991% (1100/1158)
Step: 386 | Loss: 0.174 |Combined Acc: 95.004% (1103/1161)
Step: 387 | Loss: 0.174 |Combined Acc: 95.017% (1106/1164)
Step: 388 | Loss: 0.173 |Combined Acc: 95.030% (1109/1167)
Step: 389 | Loss: 0.173 |Combined Acc: 95.043% (1112/1170)
Step: 390 | Loss: 0.173 |Combined Acc: 95.055% (1115/1173)
Step: 391 | Loss: 0.172 |Combined Acc: 95.068% (1118/1176)
Step: 392 | Loss: 0.172 |Combined Acc: 95.081% (1121/1179)
Step: 393 | Loss: 0.171 |Combined Acc: 95.093% (1124/1182)
Step: 394 | Loss: 0.171 |Combined Acc: 95.105% (1127/1185)
Step: